In [1]:
import warnings                            # This block prevents display of harmless warnings, but should be
warnings.filterwarnings('ignore')          # commented out till the final version, to avoid missing "real" warnings 
import kilroy_was_here                     # Mandatory. Allow access to shared python code in the upper 'lib' directory
from audace.audiodataset import AudioDataset  # Class for audio dataset handling

# Dataset name is the master key for dataset adressing
DATASET_NAME = 'SMALL1005'

# Initialize Dataset Object. By NOT providing a source path, you implicitly express
# express the intent of RETRIEVING an existing dataset rather than CREATING a new one
ds = AudioDataset(DATASET_NAME)

ds.info()

[2020-08-09/22:27:11.647|13.9%|80.7%|0.26GB] ------------------------------------------------------
[2020-08-09/22:27:11.647|00.0%|80.7%|0.26GB] DATASET PATH          : D:\Jupyter\ShowBees\datasets\SMALL1005
[2020-08-09/22:27:11.647|00.0%|80.7%|0.26GB] DATASET DB PATH       : D:\Jupyter\ShowBees\datasets\SMALL1005\database.db
[2020-08-09/22:27:11.647|00.0%|80.7%|0.26GB] DATASET SAMPLES PATH  : D:\Jupyter\ShowBees\datasets\SMALL1005\samples
[2020-08-09/22:27:11.647|00.0%|80.7%|0.26GB] NB SOURCE AUDIO FILES : 4
[2020-08-09/22:27:11.648|00.0%|80.7%|0.26GB] SAMPLE RATE           : 22050
[2020-08-09/22:27:11.648|00.0%|80.7%|0.26GB] DURATION              : 1.0
[2020-08-09/22:27:11.648|00.0%|80.7%|0.26GB] OVERLAP               : 0.5
[2020-08-09/22:27:11.648|00.0%|80.7%|0.26GB] NB AUDIO CHUNKS       : 4744
[2020-08-09/22:27:11.648|00.0%|80.7%|0.26GB] ------------------------------------------------------


In [2]:
from audace import providers
print(ds.listAttributes()) 
ds.addAttribute('prout')
ds.setAttribute('prout', providers.FromString('hello'))
print(ds.listAttributes())  
ds.dropAttribute('prout')
print(ds.listAttributes())   


['hive']
['hive', 'prout']
['hive']


In [3]:
ds.addFeature('MFCC')

False

In [15]:
import kilroy_was_here
from audace.jupytools import mooltipath
from audace import featurizers
from audace import providers

samples_path = mooltipath('datasets/SMALL1005/samples')
ds.setFeature('MFCC', providers.FromSample(samples_path, featurizers.MFCC(n_mfcc=20)))

4744

In [16]:
import sqlite3
import numpy as np
import kilroy_was_here
from audace.jupytools import mooltipath
from contextlib import closing

with closing(sqlite3.connect(ds.db_path, detect_types=sqlite3.PARSE_DECLTYPES)) as db:
    c = db.cursor()
    c.execute('SELECT MFCC from samples WHERE rowid = 1')
    row = c.fetchone()
    print(row[0])


[[-4.08871643e+02 -4.08703613e+02 -4.09220825e+02 -4.09472351e+02
  -4.16710815e+02 -4.13254456e+02 -4.09786530e+02 -4.14447174e+02
  -4.12743530e+02 -4.07243958e+02 -4.09594971e+02 -4.06865112e+02
  -3.93488007e+02 -3.98798035e+02 -4.07280121e+02 -4.08431427e+02
  -4.13553284e+02 -4.15519470e+02 -4.03156952e+02 -4.06555389e+02
  -4.11792206e+02 -4.01878296e+02 -4.04597229e+02 -4.11465698e+02
  -4.09013611e+02 -4.05051514e+02 -4.03041809e+02 -4.03477600e+02
  -4.02253540e+02 -3.95140228e+02 -4.04572235e+02 -4.10582886e+02
  -4.07057892e+02 -4.08725983e+02 -4.09886810e+02 -4.08797058e+02
  -4.16324005e+02 -4.23337158e+02 -4.18423279e+02 -4.18407715e+02
  -4.18167938e+02 -4.10862427e+02 -4.10666595e+02 -4.21104126e+02]
 [ 1.08411545e+02  1.18392456e+02  1.30889923e+02  1.28584534e+02
   1.27076248e+02  1.27815781e+02  1.28507874e+02  1.27841110e+02
   1.36058105e+02  1.38691833e+02  1.35923965e+02  1.32243225e+02
   1.20556290e+02  1.16201500e+02  1.20469086e+02  1.21821564e+02
   1.3071